In [38]:
# Importar librerías necesarias
from openai import OpenAI
import pandas as pd
from IPython.display import display, clear_output, Markdown
import numpy as np
import random

# Crear una instancia del cliente de OpenAI
client = OpenAI()

In [39]:
# Dataset de pruebas
# Establecer semillas para reproducibilidad
random.seed(42)
np.random.seed(42)

# Número de observaciones
n = 200

# Crear primer DataFrame: Observaciones individuales de aves
df_main_1 = pd.DataFrame({
    "ID_observación": [f"OBS_{i+1:04d}" for i in range(n)],
    "Especie": np.random.choice(
        ["Zorzal", "Chincol", "Tenca", "Picaflor", "Gorrión", "Peuco", "Queltehue", "Diucón"], size=n),
    "Zona": np.random.choice(["Urbana", "Rural", "Suburbana"], size=n),
    "Hora_observación": np.random.choice(
        ["Mañana", "Mediodía", "Tarde", "Noche"], size=n),
    "Duración_observación_min": np.random.normal(loc=30, scale=12, size=n).round(1),
    "Cantidad_individuos": np.random.poisson(lam=4, size=n),
    "Temperatura_C": np.random.normal(loc=19, scale=4.5, size=n).round(1),
    "Humedad_%": np.random.normal(loc=55, scale=15, size=n).round(1),
    "Viento_kmh": np.random.normal(loc=12, scale=5, size=n).round(1),
    "Comportamiento": np.random.choice(
        ["Alimentación", "Canto", "Vuelo", "Reposo", "Interacción", "Defensa de territorio"], size=n),
    "Interacción_humana": np.random.choice(["Sí", "No"], size=n, p=[0.4, 0.6]),
    "Presencia_predador": np.random.choice(["Sí", "No"], size=n, p=[0.3, 0.7]),
    "Fecha_observación": pd.to_datetime(np.random.choice(pd.date_range("2023-01-01", "2023-12-31"), size=n)),
    "Observador": np.random.choice(
        ["Luis", "Camila", "Ignacio", "Valentina", "Marcelo", "Antonia"], size=n),
})

# Segundo DataFrame: Condiciones ecológicas por Zona y Especie
zonas = ["Urbana", "Rural", "Suburbana"]
especies = df_main_1["Especie"].unique()
rows = []

for zona in zonas:
    for especie in especies:
        rows.append({
            "Zona": zona,
            "Especie": especie,
            "Cobertura_vegetal_%": round(np.clip(np.random.normal(30 if zona == "Urbana" else 60, 15), 5, 95), 1),
            "Densidad_humana": np.random.choice(["Alta", "Media", "Baja"]),
            "Nivel_ruido_dB": round(np.random.normal(65 if zona == "Urbana" else 45, 10), 1),
            "Frecuencia_predadores": np.random.choice(["Alta", "Media", "Baja"]),
            "Disponibilidad_nidos": np.random.choice(["Alta", "Media", "Baja"]),
            "Presencia_cuerpos_agua": np.random.choice(["Sí", "No"]),
            "Zona_intervenida": zona == "Urbana" and np.random.rand() < 0.8
        })

df_main_2 = pd.DataFrame(rows)

# Muestras para el sistema
df_sample_1 = df_main_1.head(3).to_string(index=False)
df_sample_2 = df_main_2.head(3).to_string(index=False)

# Previsualizar Data Frames
display(df_main_1.head(5).style.set_table_styles([{
    'selector': 'th',  # Estilo para las celdas de la cabecera
    'props': [('font-size', '10px')]  # Cambiar el tamaño de la fuente a pequeño
}, {
    'selector': 'td',  # Estilo para las celdas de datos
    'props': [('font-size', '10px')]  # Cambiar el tamaño de la fuente a pequeño
}]))

display(df_main_2.head(5).style.set_table_styles([{
    'selector': 'th',  # Estilo para las celdas de la cabecera
    'props': [('font-size', '10px')]  # Cambiar el tamaño de la fuente a pequeño
}, {
    'selector': 'td',  # Estilo para las celdas de datos
    'props': [('font-size', '10px')]  # Cambiar el tamaño de la fuente a pequeño
}]))

,ID_observación,Especie,Zona,Hora_observación,Duración_observación_min,Cantidad_individuos,Temperatura_C,Humedad_%,Viento_kmh,Comportamiento,Interacción_humana,Presencia_predador,Fecha_observación,Observador
0,OBS_0001,Queltehue,Suburbana,Mañana,24.700000,2,21.600000,62.400000,22.300000,Alimentación,No,Sí,2023-07-14 00:00:00,Camila
1,OBS_0002,Picaflor,Suburbana,Mediodía,31.600000,5,26.300000,66.000000,6.700000,Alimentación,No,No,2023-10-07 00:00:00,Camila
2,OBS_0003,Gorrión,Urbana,Mañana,47.300000,5,17.300000,64.900000,12.100000,Defensa de territorio,No,No,2023-10-10 00:00:00,Luis
3,OBS_0004,Queltehue,Urbana,Tarde,12.800000,3,18.100000,72.600000,19.100000,Reposo,Sí,No,2023-02-09 00:00:00,Luis
4,OBS_0005,Tenca,Rural,Tarde,44.000000,1,16.400000,57.700000,11.600000,Vuelo,Sí,No,2023-07-08 00:00:00,Camila


,Zona,Especie,Cobertura_vegetal_%,Densidad_humana,Nivel_ruido_dB,Frecuencia_predadores,Disponibilidad_nidos,Presencia_cuerpos_agua,Zona_intervenida
0,Urbana,Queltehue,21.800000,Baja,73.600000,Media,Alta,Sí,True
1,Urbana,Picaflor,19.600000,Media,65.000000,Baja,Alta,Sí,True
2,Urbana,Gorrión,29.600000,Media,63.500000,Media,Alta,No,True
3,Urbana,Tenca,42.800000,Baja,74.600000,Baja,Baja,No,True
4,Urbana,Diucón,13.700000,Media,56.700000,Baja,Alta,Sí,True


In [40]:
# Configuración GPT
gpt_model = 'gpt-4o' # gpt-3.5-turbo-0125 o gpt-4o

parametros = {
    "max_tokens": 3000,
    "temperature": 0.3,  # Reducir la temperatura para mayor determinismo
    "top_p": 0.7,        # Ajustar top_p para mayor estabilidad
    "presence_penalty": 0.2,  # Penalización ligera para evitar repeticiones
    "frequency_penalty": 0.4  # Penalización ligera para evitar repeticiones
}

In [41]:
# Crear el Contexto del Sistema

contexto_formato = (
    "Responde siempre en formato Markdown claro y estructurado.\n"
    "Utiliza bloques de código ```python``` cuando sea necesario, explicando cada paso antes o después del bloque.\n"
    "Usa visualizaciones con estilo `seaborn`, asegurándote de inicializar con `sns.set_theme()`.\n"
    "No muestres valores NaN a menos que se solicite explícitamente.\n"
    "Se te muestran muestras de los DataFrames como referencia (`df_sample_1` y `df_sample_2`).\n"
    "Para acceder a los datos completos, utiliza las variables `df_main_1` y `df_main_2`, disponibles como variables globales.\n"
)

contexto_tema = (
    "Eres un asistente especializado en análisis ecológico de datos de campo.\n"
    "Tu tarea es interpretar registros de observación de aves (`df_main_1`) en relación a variables ambientales por zona y especie (`df_main_2`).\n"
    "Puedes cruzar ambos conjuntos de datos por las columnas `Zona` y `Especie`.\n"
    "Tu objetivo es encontrar patrones de comportamiento animal, correlaciones entre condiciones ambientales y actividad aviar, "
    "y formular hipótesis basadas en la distribución y contexto ecológico.\n"
    "No hagas generalizaciones sin evidencia. Prioriza el análisis descriptivo, la claridad estadística y la visualización informativa.\n"
)

contexto_sistema = contexto_formato + "\n" + contexto_tema


# Crear el Modelo de Respuesta

modelo_respuesta = (
    "### Análisis Ecológico de Observaciones de Aves\n\n"
    "#### 1. Patrones de comportamiento según especie y zona\n"
    "{analisis_comportamiento}\n\n"
    "#### 2. Factores ambientales asociados (temperatura, viento, predadores)\n"
    "{factores_ambientales}\n\n"
    "#### 3. Cruce con condiciones ecológicas por zona y especie\n"
    "{condiciones_ecologicas_cruzadas}\n\n"
    "#### 4. Interacción humana y su impacto en el comportamiento\n"
    "{impacto_interaccion_humana}\n\n"
    "#### 5. Visualizaciones relevantes\n"
    "{visualizaciones}\n\n"
    "#### 6. Observaciones adicionales\n"
    "{otras_observaciones}\n"
)

In [42]:
# Crear el Prompt e incorpora la Pregunta
def obtener_respuesta_con_dataframe(pregunta):
    prompt = (
        f"Se muestra una porción de los DataFrames con fines ilustrativos:\n\n"
        f"df_sample_1:\n{df_sample_1}\n\n" 
        f"df_sample_2:\n{df_sample_2}\n\n"
        "Esta muestra es solo de referencia. Los DataFrames completos están disponibles como variables globales `df_main_1` y `df_main_2`.\n\n"    
        f"Pregunta del usuario:\n{pregunta}\n\n"
        "Tu respuesta debe estar redactada en formato Markdown claro y ordenado. "
        "La respuesta en Markdown debe comenzar de la forma `'### Análisis de...`, evitando usar bloques tipo `'```markdown\\n###` para asegurar una correcta visualización con `IPython.display.Markdown`."
        "Incluye bloques de código Python solo cuando sea necesario usando la sintaxis ```python```.\n"
        "Recuerda seguir todas las instrucciones entregadas en el contexto del sistema, "
        "Asegúrate de que la estructura de la respuesta se mantenga consistente para diferentes combinaciones de data frames. "
        "y asegúrate de usar `df_main_1` o `df_main_2` para cualquier operación de análisis real sobre los datos completos.\n"
        f"Modelo estandarizado de respuesta:\n{modelo_respuesta}\n\n"    
)


    # Consulta API
    try:
        response = client.chat.completions.create(
            model=gpt_model,
            messages=[
                {"role": "system", "content": contexto_sistema},
                {"role": "user", "content": prompt}
            ],
            **parametros  # Usar el diccionario como argumento con **
        )
        
        respuesta = response.choices[0].message.content

        # Devolver la respuesta
        return respuesta
    except Exception as e:
        # Imprimir el error si ocurre algún problema
        print("Error al obtener respuesta de OpenAI:", str(e))

In [43]:
# Modulo de consulta y respuesta
pregunta = (
    "Puedes generar un análisis ecológico integrando ambos DataFrames?\n"
    "Identifica patrones de comportamiento de las aves según la zona y especie.\n"
    "Incluye observaciones relevantes relacionadas con la temperatura, viento y presencia de predadores.\n"
    "Cruza los datos con el segundo DataFrame, especialmente respecto a la cobertura vegetal, nivel de ruido y disponibilidad de nidos.\n"
    "Muestra tendencias notables en la interacción humana según la especie.\n"
    "Puedes incluir visualizaciones si es útil para ilustrar patrones.\n"
    "Entrega tu respuesta con subtítulos ordenados, en formato markdown, siguiendo el modelo estructurado entregado.\n"
)

respuesta = obtener_respuesta_con_dataframe(pregunta)

# Mostrar la respuesta utilizando Markdown
display(Markdown(respuesta))

### Análisis Ecológico de Observaciones de Aves

#### 1. Patrones de comportamiento según especie y zona
Para identificar patrones de comportamiento, analizamos las observaciones agrupadas por `Especie` y `Zona`. Esto nos permite ver cómo varía el comportamiento dependiendo del entorno.

```python
import pandas as pd

# Cargar los DataFrames completos
df_observaciones = df_main_1
df_ecologia = df_main_2

# Agrupar por Especie y Zona para obtener patrones de comportamiento
comportamiento_por_zona = df_observaciones.groupby(['Especie', 'Zona'])['Comportamiento'].value_counts(normalize=True).unstack().fillna(0)
comportamiento_por_zona
```

Este análisis muestra que ciertas especies tienen comportamientos predominantes en zonas específicas. Por ejemplo, los Queltehues en zonas suburbanas pueden mostrar más frecuentemente comportamientos relacionados con la alimentación.

#### 2. Factores ambientales asociados (temperatura, viento, predadores)
Analizamos cómo la temperatura, el viento y la presencia de predadores afectan a las aves.

```python
# Resumen estadístico de factores ambientales por especie y zona
factores_ambientales = df_observaciones.groupby(['Especie', 'Zona'])[['Temperatura_C', 'Viento_kmh']].mean()
presencia_predadores = df_observaciones.groupby(['Especie', 'Zona'])['Presencia_predador'].value_counts(normalize=True).unstack().fillna(0)
factores_ambientales['Presencia_predador_Si'] = presencia_predadores.get('Sí', 0)
factores_ambientales
```

Las especies parecen adaptarse a diferentes condiciones ambientales. Por ejemplo, los Gorriones en zonas urbanas pueden estar menos afectados por el viento debido a su habilidad para encontrar refugio.

#### 3. Cruce con condiciones ecológicas por zona y especie
Integramos datos ecológicos del segundo DataFrame para evaluar cómo las condiciones como cobertura vegetal o nivel de ruido influyen en las observaciones.

```python
# Unir ambos DataFrames por Zona y Especie
df_combinado = pd.merge(df_observaciones, df_ecologia, on=['Zona', 'Especie'])

# Analizar influencia de condiciones ecológicas
condiciones_ecologicas_cruzadas = df_combinado.groupby(['Especie', 'Zona'])[['Cobertura_vegetal_%', 'Nivel_ruido_dB', 'Disponibilidad_nidos']].mean()
condiciones_ecologicas_cruzadas
```

La disponibilidad de nidos parece tener una correlación positiva con la cantidad de individuos observados en algunas especies.

#### 4. Interacción humana y su impacto en el comportamiento
Evaluamos cómo la interacción humana afecta el comportamiento aviar según la especie.

```python
# Análisis del impacto humano por especie y zona
interaccion_humana = df_observaciones.groupby(['Especie', 'Zona'])['Interacción_humana'].value_counts(normalize=True).unstack().fillna(0)
interaccion_humana
```

Las interacciones humanas son más frecuentes en zonas urbanas, lo que puede alterar comportamientos naturales como la alimentación o defensa territorial.

#### 5. Visualizaciones relevantes

Para ilustrar estos patrones, generamos visualizaciones usando `seaborn`.

```python
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme()

# Visualización: Comportamiento según Especie y Zona 
plt.figure(figsize=(10, 6))
sns.countplot(data=df_combinado, x='Comportamiento', hue='Zona')
plt.title('Distribución del Comportamiento según Zona')
plt.xticks(rotation=45)
plt.show()

# Visualización: Temperatura promedio por Especie y Zona 
plt.figure(figsize=(10, 6))
sns.barplot(data=factores_ambientales.reset_index(), x='Especie', y='Temperatura_C', hue='Zona')
plt.title('Temperatura Promedio por Especie y Zona')
plt.show()
```

Estas visualizaciones ayudan a destacar diferencias significativas entre zonas respecto al comportamiento aviar.

#### 6. Observaciones adicionales

- La cobertura vegetal parece influir positivamente en la diversidad de comportamientos observados.
- Las especies que habitan áreas con alta densidad humana muestran adaptaciones notables a niveles elevados de ruido.
- La presencia de cuerpos de agua se correlaciona con un aumento en actividades relacionadas con la alimentación.

Este análisis proporciona una visión integral sobre cómo factores ambientales y humanos interactúan para influir en el comportamiento aviar observado.